# Possible Future Explorations: Items and Runes

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf   
## if error: python3.6 -m pip install scipy==1.2 --upgrade
import json 
import pprint
import requests
import re

In [2]:
folder = "NA"
region = "na"

In [3]:
df = pd.read_csv('data/{}/{}clean.csv'.format(folder,region),low_memory=False)
print(df.shape)

(29473, 1186)


In [4]:
### Participants 0 - 4 are on team 0
all(df["teams_0_win"] == df["participants_4_stats_win"])

True

In [5]:
champions = df[["teams_0_win", 
                "participants_0_championId", "participants_1_championId", 
                "participants_2_championId", "participants_3_championId", 
                "participants_4_championId", "participants_5_championId", 
                "participants_6_championId", "participants_7_championId",
                "participants_8_championId", "participants_9_championId",
                "participants_0_stats_win", "participants_1_stats_win", 
                "participants_2_stats_win", "participants_3_stats_win",
                "participants_4_stats_win", "participants_5_stats_win",
                "participants_6_stats_win", "participants_7_stats_win",
                "participants_8_stats_win","participants_9_stats_win",
                "participants_0_timeline_lane", "participants_1_timeline_lane", 
                "participants_2_timeline_lane", "participants_3_timeline_lane", 
                "participants_4_timeline_lane", "participants_5_timeline_lane", 
                "participants_6_timeline_lane", "participants_7_timeline_lane",
                "participants_8_timeline_lane", "participants_9_timeline_lane",]].copy()

In [6]:
champions

,teams_0_win,participants_0_championId,participants_1_championId,participants_2_championId,participants_3_championId,participants_4_championId,participants_5_championId,participants_6_championId,participants_7_championId,participants_8_championId,...,participants_0_timeline_lane,participants_1_timeline_lane,participants_2_timeline_lane,participants_3_timeline_lane,participants_4_timeline_lane,participants_5_timeline_lane,participants_6_timeline_lane,participants_7_timeline_lane,participants_8_timeline_lane,participants_9_timeline_lane
0,1,Fizz,Karma,Lucian,Kindred,Ornn,Pyke,Alistar,Gangplank,Nidalee,...,MIDDLE,BOTTOM,MIDDLE,JUNGLE,TOP,BOTTOM,BOTTOM,MIDDLE,JUNGLE,TOP
1,0,Volibear,Ezreal,Braum,Hecarim,TwistedFate,Renekton,Kalista,Taric,Nidalee,...,TOP,MIDDLE,TOP,JUNGLE,TOP,TOP,BOTTOM,BOTTOM,JUNGLE,MIDDLE
2,0,Kayn,Katarina,Senna,Bard,Jayce,Irelia,Blitzcrank,Ezreal,Shaco,...,JUNGLE,MIDDLE,BOTTOM,BOTTOM,TOP,TOP,BOTTOM,BOTTOM,JUNGLE,TOP
3,0,Leblanc,Ezreal,Kindred,Karma,Maokai,RekSai,Ryze,Yasuo,Taric,...,MIDDLE,MIDDLE,JUNGLE,MIDDLE,TOP,JUNGLE,MIDDLE,TOP,BOTTOM,BOTTOM
4,1,Ryze,Aphelios,Leblanc,JarvanIV,Galio,Ekko,Qiyana,Irelia,Jhin,...,TOP,MIDDLE,MIDDLE,JUNGLE,BOTTOM,JUNGLE,MIDDLE,TOP,BOTTOM,BOTTOM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29468,1,Soraka,Cassiopeia,Thresh,MasterYi,Zed,Pantheon,Khazix,Lulu,Caitlyn,...,BOTTOM,TOP,BOTTOM,JUNGLE,TOP,MIDDLE,JUNGLE,BOTTOM,BOTTOM,MIDDLE
29469,1,Ezreal,Fiora,Galio,Lux,Graves,Zac,Camille,Taliyah,Caitlyn,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
29470,1,Kalista,Shyvana,Fiora,Pantheon,Cassiopeia,Elise,MissFortune,Darius,Karma,...,MIDDLE,JUNGLE,TOP,BOTTOM,BOTTOM,JUNGLE,BOTTOM,TOP,MIDDLE,BOTTOM
29471,1,Ezreal,Karma,Sylas,Camille,Fiddlesticks,Heimerdinger,Renekton,Varus,Zac,...,BOTTOM,BOTTOM,MIDDLE,JUNGLE,JUNGLE,TOP,TOP,BOTTOM,JUNGLE,BOTTOM


In [7]:
#series of total games per champ
champgames = []
for col in champions.columns:
    champgames.append(champions[col].value_counts())

totalgames = champgames[1]
for i in range(2,10):
    totalgames += champgames[i]

In [8]:
#series of win percentages per champ
champwins = {}
for i in range(0,5):
    champwins[i] = champions.groupby('participants_{}_championId'.format(i)
                                    ).mean()['teams_0_win']
for i in range(5,10):
    champwins[i] = 1 - champions.groupby('participants_{}_championId'.format(i)
                                        ).mean()['teams_0_win']    
totalwins = pd.DataFrame(champwins).mean(axis=1)

In [9]:
#create df
wins = totalwins.to_frame().join(totalgames.to_frame())
wins.columns = ['win_percent','num_games']
wins

,win_percent,num_games
Aatrox,0.465573,2146
Ahri,0.503014,964
Akali,0.468735,1864
Alistar,0.486304,1344
Amumu,0.417434,215
...,...,...
Zed,0.500925,2584
Ziggs,0.477372,683
Zilean,0.477617,758
Zoe,0.520933,2060


In [10]:
summoner = requests.get("http://ddragon.leagueoflegends.com/cdn/10.15.1/data/en_US/summoner.json").json()

In [11]:
items = requests.get("http://ddragon.leagueoflegends.com/cdn/10.15.1/data/en_US/item.json").json()

In [12]:
items['data']['1011']

{'name': "Giant's Belt",
 'description': '<stats>+380 Health</stats>',
 'colloq': ';',
 'plaintext': 'Greatly increases Health',
 'from': ['1028'],
 'into': ['3083', '3084', '3022', '3143', '3742'],
 'image': {'full': '1011.png',
  'sprite': 'item0.png',
  'group': 'item',
  'x': 144,
  'y': 0,
  'w': 48,
  'h': 48},
 'gold': {'base': 600, 'purchasable': True, 'total': 1000, 'sell': 700},
 'tags': ['Health'],
 'maps': {'11': True, '12': True, '21': True, '22': False},
 'stats': {'FlatHPPoolMod': 380},
 'depth': 2}

In [13]:
player = pd.read_csv('data/{}/{}player.csv'.format(folder,region),
                     low_memory=False)
print(df.shape)

(29473, 1186)


In [14]:
player[[col for col in player.columns if 'perk' in col]]

,stats_perk0,stats_perk2Var3,stats_perk5,stats_perk2,stats_perk5Var3,stats_perk2Var2,stats_perk5Var1,stats_perk5Var2,stats_perk4Var3,stats_perk3Var2,...,stats_perk3,stats_perk0Var3,stats_perk1Var2,stats_perk1Var1,stats_perk0Var1,stats_perk1,stats_perkSubStyle,stats_perk0Var2,stats_perk4Var2,stats_perk3Var1
0,8112,0,8014,8138,0,0,921,0,0,5,...,8135,0,0,1011,1848,8143,8000,0,480,4590
1,8005,0,8242,9105,0,30,0,0,0,0,...,8299,471,120,674,1088,9111,8400,616,0,557
2,8010,0,8143,9105,0,30,510,0,0,0,...,8299,0,300,1089,817,9111,8100,0,5,850
3,8112,0,8210,8138,0,0,0,0,0,4,...,8135,0,0,793,1033,8139,8200,0,828,2168
4,8230,0,8135,8210,0,0,4425,4,0,0,...,8237,0,1060,250,17,8226,8100,0,0,382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294725,8010,0,8352,9104,843,0,230,1410,0,0,...,8014,0,0,0,145,9111,8300,0,0,66
294726,8439,0,8347,8444,0,0,0,0,0,0,...,8242,0,0,0,84,8446,8300,0,0,0
294727,8214,0,8453,8233,0,50,697,478,0,0,...,8236,0,321,250,720,8226,8400,1721,0,24
294728,8229,0,8347,8210,0,0,0,0,0,0,...,8237,0,486,250,2256,8226,8300,0,0,736
